In [ ]:
# --------------------------------------------------
# 0. Instalación de Paquetes (ejecutar si es necesario)
# --------------------------------------------------
#=
import Pkg

# Paquetes estándar de datos y estadísticas
Pkg.add(["CSV", "DataFrames", "StatsModels", "GLM", "Random", "Downloads", "CategoricalArrays", "Statistics", "PrettyTables"])

# Paquetes de Machine Learning (MLJ)
Pkg.add("MLJ")
Pkg.add("MLJScikitLearnInterface") # Para OLS, Lasso, RF
Pkg.add("MLJFlux") # Para Redes Neuronales
Pkg.add("Flux")
=#


# --------------------------------------------------
# 1. Carga de Paquetes
# --------------------------------------------------
using CSV, DataFrames, StatsModels, GLM, Random, Downloads, CategoricalArrays, Statistics, PrettyTables
using MLJ, MLJScikitLearnInterface, MLJFlux, Flux

println("Paquetes cargados.")


# --------------------------------------------------
# 2. Carga y Limpieza de Datos (Parte I)
# --------------------------------------------------

# --- 2.1 Cargar Datos ---
nombres = [
    "abdt", "tg", "inuidur1", "inuidur2", "female", "black", "hispanic", 
    "othrace", "dep", "q1", "q2", "q3", "q4", "q5", "q6", "recall", 
    "agelt35", "agegt54", "durable", "nondurable", "lusd", "husd", "muld"
]

url = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/penn_jae.dat"
file_path = "penn_jae.dat"
if !isfile(file_path)
    Downloads.download(url, file_path)
end

df = CSV.read(file_path, DataFrame, skipto=2, header=nombres, delim=' ', ignorerepeated=true)
println("Datos originales: ", size(df))

# --- 2.2 Limpieza y Configuración ---

# I. Mantener 'tg' == 0 o 4
df_cleaned = filter(row -> row.tg == 0 || row.tg == 4, df)
println("Datos después de filtrar 'tg': ", size(df_cleaned))

# II. Definir 'T4' (d)
df_cleaned[!, :T4] = (df_cleaned.tg .== 4)

# III. Definir 'y' como log('inuidur1')
df_cleaned[!, :y] = log.(Float64.(df_cleaned.inuidur1))

# Reemplazar -Inf (log(0)) con 'missing'
df_cleaned.y = replace(df_cleaned.y, -Inf => missing)

# Eliminar filas con 'missing' en 'y'
dropmissing!(df_cleaned, [:y])
println("Datos después de eliminar 'missing' en y: ", size(df_cleaned))

# IV. Crear dummies 'dep_1', 'dep_2' (dep_0 es referencia)
df_cleaned[!, :dep_1] = (df_cleaned.dep .== 1)
df_cleaned[!, :dep_2] = (df_cleaned.dep .== 2)

# V. Definir lista de features 'x'
feature_list = [
    :female, :black, :othrace,
    :dep_1, :dep_2,  # :dep_0 es referencia
    :q2, :q3, :q4, :q5, :q6, # :q1 es referencia
    :recall, :agelt35, :agegt54,
    :durable, :nondurable, :lusd, :husd
]

# Definir x, y, d
x = df_cleaned[!, feature_list]
y = df_cleaned.y
d = Float64.(df_cleaned.T4) # Convertir Boolean a Float64

# Coercionar 'x' para MLJ (tratar todo como Continuo)
x = coerce(x, Count => Continuous)

println("Configuración completa.")
println(first(x, 5))


# --------------------------------------------------
# 3. Definición de Funciones DML
# --------------------------------------------------

# --- 3.1 Cargar todos los modelos de MLJ --- 
LinearRegressor = @load LinearRegressor pkg=MLJScikitLearnInterface verbosity=0
LassoCVRegressor = @load LassoCVRegressor pkg=MLJScikitLearnInterface verbosity=0
RandomForestRegressor = @load RandomForestRegressor pkg=MLJScikitLearnInterface verbosity=0
NeuralNetworkRegressor = @load NeuralNetworkRegressor pkg=MLJFlux verbosity=0 

LogisticClassifier = @load LogisticRegressionCV pkg=MLJScikitLearnInterface verbosity=0
# Usamos LogisticRegressionCV para Lasso (L1)
LassoClassifier = @load LogisticRegressionCV pkg=MLJScikitLearnInterface verbosity=0 
RandomForestClassifier = @load RandomForestClassifier pkg=MLJScikitLearnInterface verbosity=0
NeuralNetworkClassifier = @load NeuralNetworkClassifier pkg=MLJFlux verbosity=0

# --- 3.2 Función auxiliar del laboratorio para los pliegues ---
function training_sample_append(cv_split, test_sample_index)
        training_indices = []
        for vector in cv_split[Not(test_sample_index)]
                training_indices = [training_indices; vector]
        end
        return training_indices, cv_split[test_sample_index]
end

# --- 3.3 Función DML (Parte II) ---
function dml(x, d, y, modely, modeld, nfold; classifier=true)
        n = length(y)
        cv = [partition(eachindex(y), fill(1/nfold, nfold-1)..., shuffle = true, rng = 1234)...]
        
        # Coercionar variables objetivo para MLJ
        y_mlj = y
        d_mlj = classifier ? categorical(d) : d # d es categórico para clasificadores

        machine_y = machine(modely, x, y_mlj, scitype_check_level=0)
        machine_d = machine(modeld, x, d_mlj, scitype_check_level=0)
        
        y_hat = zeros(n)
        d_hat = zeros(n)

        for fold in 1:nfold
                training_fold, test_fold = training_sample_append(cv, fold)
                
                # Entrenar y predecir Y (Resultado)
                MLJ.fit!(machine_y, rows = training_fold)
                y_hat[test_fold] = MLJ.predict(machine_y, x[test_fold, :])

                # Entrenar y predecir D (Tratamiento)
                MLJ.fit!(machine_d, rows = training_fold)
                if classifier
                    d_hat_probs = MLJ.predict(machine_d, x[test_fold, :])
                    # Obtener la probabilidad de la clase '1.0' (equivalente a predict_proba[:, 1])
                    d_hat[test_fold] = pdf.(d_hat_probs, 1.0)
                else
                    d_hat[test_fold] = MLJ.predict(machine_d, x[test_fold, :])
                end
        end

        # Regresión final: resy ~ resd
        resy = y .- y_hat
        resd = reshape(d .- d_hat, (n, 1))
        
        ols_data = DataFrame(resy = resy, resd = resd[:, 1])
        estimate_model = lm(@formula(resy ~ resd), ols_data)
        
        # Extraer coeficiente y error estándar para 'resd' (el segundo coeficiente)
        coef_est = GLM.coef(estimate_model)[2]
        se = GLM.stderror(estimate_model)[2]
        
        println(" coef (se) = ", coef_est , " (", se, ")")
        return coef_est, se, resy, resd
end

# --- 3.4 Función DML Naive (Parte III) ---
function dml_naive(x, d, y, modely, modeld; classifier=true)
        n = length(y)
        
        y_mlj = y
        d_mlj = classifier ? categorical(d) : d

        # 1. Entrenar modely en TODOS los datos y predecir EN MUESTRA
        machine_y = machine(modely, x, y_mlj, scitype_check_level=0)
        MLJ.fit!(machine_y, rows = 1:n)
        y_hat = MLJ.predict(machine_y, x)

        # 2. Entrenar modeld en TODOS los datos y predecir EN MUESTRA
        machine_d = machine(modeld, x, d_mlj, scitype_check_level=0)
        MLJ.fit!(machine_d, rows = 1:n)
        
        d_hat = zeros(n)
        if classifier
            d_hat_probs = MLJ.predict(machine_d, x)
            d_hat = pdf.(d_hat_probs, 1.0)
        else
            d_hat = MLJ.predict(machine_d, x)
        end
        
        # 3. Calcular residuales (en muestra)
        resy = y .- y_hat
        resd = reshape(d .- d_hat, (n, 1))

        # 4. Regresión final
        ols_data = DataFrame(resy = resy, resd = resd[:, 1])
        estimate_model = lm(@formula(resy ~ resd), ols_data)
        
        coef_est = GLM.coef(estimate_model)[2]
        se = GLM.stderror(estimate_model)[2]
        
        println(" coef (se) = ", coef_est , " (", se, ")")
        return coef_est, se, resy, resd
end

# --- 3.5 Función de Resumen --- 
function summarize(point, stderr, resy, resd, name)
        return DataFrame(
                model = [name],
                estimate = [point], stderr = [stderr], 
                rmse_y = [sqrt(mean(resy .^ 2))],
                # Asegurarse que resd sea un vector para el cálculo de la media
                rmse_d = [sqrt(mean(vec(resd) .^ 2))]
        )
end

println("Funciones DML definidas.")


# --------------------------------------------------
# 4. Parte II: Ejecutar Debiased ML (con Cross-Fitting)
# --------------------------------------------------

# --- 4.1 Definir Modelos --- 

# 1. OLS / Logit
modely_ols = Standardizer() |> LinearRegressor()
modeld_ols = Standardizer() |> LogisticClassifier(cv=5, random_state=123)

# 2. Lasso
modely_lasso = Standardizer() |> LassoCVRegressor(cv=5, random_state=123)
modeld_lasso = Standardizer() |> LassoClassifier(cv=5, penalty="l1", solver="liblinear", random_state=123)

# 3. Random Forest
# NOTA: RF no necesita estandarización previa
modely_rf = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, random_state=123)
modeld_rf = RandomForestClassifier(n_estimators=100, min_samples_leaf=5, random_state=123)

# 4. Neural Network (NN)
# Usamos los hiperparámetros de Python: (50, 20) capas, early_stopping
builder_reg = MLJFlux.MLP(hidden=(50, 20), σ=relu)
modely_nn = Standardizer() |> NeuralNetworkRegressor(
    builder = builder_reg, 
    epochs = 100, # Python usó max_iter=500, early_stopping. 100-200 epochs es un sustituto razonable.
    batch_size = 32,
    optimiser = Flux.ADAM(0.001),
    rng = 123
)

builder_clf = MLJFlux.MLP(hidden=(50, 20), σ=relu)
modeld_nn = Standardizer() |> NeuralNetworkClassifier(
    builder = builder_clf, 
    epochs = 100,
    batch_size = 32,
    optimiser = Flux.ADAM(0.001),
    rng = 123
)

println("Modelos definidos. Ejecutando DML (Cross-Fit)...")

# --- 4.2 Ejecutar DML --- 
println("\n--- OLS/Logit ---")
result_OLS = dml(x, d, y, modely_ols, modeld_ols, 10, classifier=true)
table_OLS = summarize(result_OLS..., "OLS/Logit")

println("\n--- Lasso ---")
result_Lasso = dml(x, d, y, modely_lasso, modeld_lasso, 10, classifier=true)
table_Lasso = summarize(result_Lasso..., "Lasso")

println("\n--- Random Forest ---")
result_RF = dml(x, d, y, modely_rf, modeld_rf, 10, classifier=true)
table_RF = summarize(result_RF..., "Random Forest")

println("\n--- NN (MLP) ---")
result_NN = dml(x, d, y, modely_nn, modeld_nn, 10, classifier=true)
table_NN = summarize(result_NN..., "NN (MLP)")

# --- 4.3 Combinar y Mostrar Tabla --- 
table_dml = vcat(table_OLS, table_Lasso, table_RF, table_NN)
table_dml_sorted = sort(table_dml, [:rmse_y, :rmse_d])

println("\n--- Resultados DML (Cross-Fit) --- (RMSE ordenado de menor a mayor)")
pretty_table(table_dml_sorted)


# --------------------------------------------------
# 5. Parte III: Ejecutar DML Naive (Sin Cross-Fitting)
# --------------------------------------------------

println("\nEjecutando Naive DML (Sin Cross-Fit)...")

println("\n--- OLS/Logit (Naive) ---")
result_OLS_naive = dml_naive(x, d, y, modely_ols, modeld_ols, classifier=true)
table_OLS_naive = summarize(result_OLS_naive..., "OLS/Logit")

println("\n--- Lasso (Naive) ---")
result_Lasso_naive = dml_naive(x, d, y, modely_lasso, modeld_lasso, classifier=true)
table_Lasso_naive = summarize(result_Lasso_naive..., "Lasso")

println("\n--- Random Forest (Naive) ---")
result_RF_naive = dml_naive(x, d, y, modely_rf, modeld_rf, classifier=true)
table_RF_naive = summarize(result_RF_naive..., "Random Forest")

println("\n--- NN (MLP) (Naive) ---")
result_NN_naive = dml_naive(x, d, y, modely_nn, modeld_nn, classifier=true)
table_NN_naive = summarize(result_NN_naive..., "NN (MLP)")

# --- 5.1 Combinar y Mostrar Tabla --- 
table_naive = vcat(table_OLS_naive, table_Lasso_naive, table_RF_naive, table_NN_naive)
table_naive_sorted = sort(table_naive, [:rmse_y, :rmse_d])

println("\n--- Resultados Naive DML (Sin Cross-Fit) --- (RMSE ordenado de menor a mayor)")
pretty_table(table_naive_sorted)


# --------------------------------------------------
# 6. Comparación y Preguntas de Análisis
# --------------------------------------------------

# --- 6.1 Tabla de Comparación (como se pidió en el chat) ---

# Añadir una columna 'Method' a cada tabla
table_dml_copy = copy(table_dml)
table_naive_copy = copy(table_naive)

table_dml_copy[!, :Method] .= "DML (Cross-Fit)"
table_naive_copy[!, :Method] .= "Naive (No Cross-Fit)"

# Concatenar
comparison = vcat(table_dml_copy, table_naive_copy)

# Ordenar por modelo para agruparlos
comparison_sorted = sort(comparison, :model)

println("\n--- Comparación Completa: DML vs. Naive ---")
pretty_table(comparison_sorted, header_crayon = crayon(bold = true))

# --- 6.2 Respuestas a las Preguntas ---

println("""
### Pregunta 1: What can you say about the RMSE for predicting y and d?

**Respuesta:** Al observar la tabla de comparación, se ve claramente que los valores de `rmse_y` y `rmse_d` son **consistentemente más bajos** en el método "Naive (No Cross-Fit)" que en el método "DML (Cross-Fit)". Esta diferencia es mucho más pronunciada en los modelos flexibles como Random Forest y NN.

### Pregunta 2: Why is it that estimating with one function yields lower RMSE than another?

**Respuesta:** Esto se debe al **sobreajuste (overfitting)**.

* La función **Naive (`dml_naive`)** entrena y evalúa el modelo en el *mismo conjunto de datos* (error en-muestra o *in-sample*). Los modelos complejos (especialmente RF y NN) son muy buenos para "memorizar" los datos de entrenamiento, incluido el ruido aleatorio. Esto da como resultado un RMSE artificialmente bajo y excesivamente optimista.
* La función **DML (`dml`)** usa **cross-fitting (ajuste cruzado)**. Entrena el modelo en una parte de los datos (ej. 9 pliegues) y lo evalúa en una parte *que no ha visto* (el pliegue restante). Este error (fuera-de-muestra o *out-of-sample*) es una medida mucho más honesta y realista del verdadero poder predictivo del modelo en datos nuevos.

### Pregunta 3: What problem would we have if we chose to estimate without cross-fitting?

**Respuesta:** El problema principal es el **sesgo por sobreajuste (overfitting bias)**.

La teoría de DML (Double Machine Learning) requiere que los residuos (`resy` y `resD`) se generen de una manera que sea "ortogonal" (estadísticamente independiente) del proceso de estimación. El **cross-fitting** es el mecanismo que nos permite lograr esto.

Si *no* usamos cross-fitting (como en `dml_naive`):
1.  Los modelos flexibles (RF, NN) se sobreajustarán masivamente a los datos.
2.  Los residuos `resy` y `resD` que calculamos estarán "contaminados" por este sobreajuste.
3.  Cuando ejecutamos la regresión final (`resy ~ resD`), la relación que encontramos estará **sesgada**. El estimador del efecto causal (`estimate`) no será confiable y su inferencia (el `stderr`) será inválida. 

En resumen, **sin cross-fitting, sacrificamos la validez estadística y la inferencia causal correcta por un falso sentido de precisión (un RMSE más bajo) que proviene del sobreajuste.**
""")